In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

class EngineEmissionsAnalysis:
    def __init__(self):
        # Standard conditions based on research papers
        self.std_temp = 25  # °C
        self.std_humidity = 75  # %
        self.std_pressure = 101.325  # kPa
        
    def diesel_nox_correction_pre1994(self, temperature, humidity):
        """
        NOx correction for pre-1994 naturally aspirated diesel engines
        Based on Krause et al. (1973)
        """
        return 1 + 0.001368 * (temperature - 29.444) - 0.01512 * (humidity - 10.71)
    
    def diesel_nox_correction_post1994(self, temperature, humidity):
        """
        NOx correction for post-1994 turbocharged diesel engines
        Based on newer research
        """
        return 1 + 0.00446 * (temperature - 25) - 0.018708 * (humidity - 10.71)
    
    def si_engine_correction(self, temperature, humidity, engine_type='modern'):
        """
        Correction for Spark Ignition engines
        Based on SwRI research
        """
        if engine_type == 'modern':
            # Modern engines with 3-way catalyst
            return 1 - 0.0232 * (humidity - 10.71)
        else:
            # Carbureted engines
            return 1 + 0.0022 * (temperature - 25) - 0.0280 * (humidity - 10.71)
    
    def calculate_daily_variations(self, base_temp=30, base_humidity=60):
        """
        Simulates daily temperature and humidity variations
        """
        hours = np.arange(24)
        
        # Temperature variation (sinusoidal with max at 14:00)
        temp_variation = base_temp + 8 * np.sin(np.pi * (hours - 6) / 12)
        
        # Humidity variation (inverse to temperature)
        humidity_variation = base_humidity - 20 * np.sin(np.pi * (hours - 6) / 12)
        
        return temp_variation, humidity_variation

    def plot_daily_emissions_profile(self):
        """
        Plots daily emissions profile for different engine types
        """
        temp, humid = self.calculate_daily_variations()
        
        # Calculate corrections for different engine types
        pre94_diesel = [self.diesel_nox_correction_pre1994(t, h) 
                       for t, h in zip(temp, humid)]
        post94_diesel = [self.diesel_nox_correction_post1994(t, h) 
                        for t, h in zip(temp, humid)]
        modern_si = [self.si_engine_correction(t, h, 'modern') 
                    for t, h in zip(temp, humid)]
        
        plt.figure(figsize=(12, 8))
        
        # Plot emissions corrections
        plt.subplot(2, 1, 1)
        plt.plot(range(24), pre94_diesel, label='Pre-1994 Diesel')
        plt.plot(range(24), post94_diesel, label='Post-1994 Diesel')
        plt.plot(range(24), modern_si, label='Modern SI')
        plt.xlabel('Hour of Day')
        plt.ylabel('NOx Correction Factor')
        plt.legend()
        plt.title('Daily NOx Emissions Correction Factors')
        
        # Plot temperature and humidity
        plt.subplot(2, 1, 2)
        plt.plot(range(24), temp, label='Temperature (°C)')
        plt.plot(range(24), humid, label='Humidity (%)')
        plt.xlabel('Hour of Day')
        plt.ylabel('Value')
        plt.legend()
        plt.title('Daily Temperature and Humidity Variation')
        
        plt.tight_layout()
        plt.show()

# Example usage
analyzer = EngineEmissionsAnalysis()
analyzer.plot_daily_emissions_profile()

# Additional analysis for seasonal variations
def analyze_seasonal_impact():
    seasons = {
        'Summer': {'temp': 35, 'humidity': 70},
        'Winter': {'temp': 15, 'humidity': 40},
        'Spring': {'temp': 25, 'humidity': 55},
        'Fall': {'temp': 20, 'humidity': 60}
    }
    
    results = {}
    for season, conditions in seasons.items():
        results[season] = {
            'Pre-1994 Diesel': analyzer.diesel_nox_correction_pre1994(
                conditions['temp'], conditions['humidity']),
            'Post-1994 Diesel': analyzer.diesel_nox_correction_post1994(
                conditions['temp'], conditions['humidity']),
            'Modern SI': analyzer.si_engine_correction(
                conditions['temp'], conditions['humidity'])
        }
    
    return pd.DataFrame(results)

seasonal_results = analyze_seasonal_impact()
print("\nSeasonal NOx Correction Factors:")
print(seasonal_results)